In [13]:
# Importar librerías al principio
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

# Función para cargar y balancear el dataset
def cargar_balancear_dataset(ruta_csv):
    df = pd.read_csv(ruta_csv)
    class_0 = df[df['not.fully.paid'] == 0]
    class_1 = df[df['not.fully.paid'] == 1]
    class_0_balanced = class_0.sample(len(class_1), random_state=42)
    df_balanced = pd.concat([class_0_balanced, class_1])
    X = df_balanced.drop('not.fully.paid', axis=1)
    y = df_balanced['not.fully.paid']
    return X, y

# Función para preprocesar los datos (One-Hot Encoding, estandarización y PCA)
def preprocesar_datos(X_train, X_test):
    # Hacer One-Hot Encoding de todo el conjunto de datos (entrenamiento + prueba) para mantener las mismas columnas
    X_combined = pd.concat([X_train, X_test], axis=0)
    X_combined_encoded = pd.get_dummies(X_combined, drop_first=True)

    # Separar de nuevo el conjunto de entrenamiento y prueba después del One-Hot Encoding
    X_train_encoded = X_combined_encoded.iloc[:len(X_train), :]
    X_test_encoded = X_combined_encoded.iloc[len(X_train):, :]

    # Estandarizar las características
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_encoded)
    X_test_scaled = scaler.transform(X_test_encoded)

    # Aplicar PCA
    pca = PCA(n_components=10)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)

    return X_train_pca, X_test_pca, scaler, pca

# Función para entrenar y validar el modelo LDA
def entrenar_modelo(X_train, y_train):
    lda_model = LinearDiscriminantAnalysis()
    cross_val_scores = cross_val_score(lda_model, X_train, y_train, cv=5)
    lda_model.fit(X_train, y_train)
    print(f"Rendimiento promedio (validación cruzada): {cross_val_scores.mean()}")
    return lda_model

# Función para evaluar el modelo con ajuste de umbral
def evaluar_con_umbral(modelo, X_test, y_test, umbral=0.5):
    y_probs = modelo.predict_proba(X_test)[:, 1]
    y_pred_threshold = np.where(y_probs > umbral, 1, 0)
    print(f"Evaluación con umbral = {umbral}")
    print(classification_report(y_test, y_pred_threshold))


# Ejecución del código

# Cargar y balancear el dataset
X, y = cargar_balancear_dataset('./Clasificacion_banco.csv')

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Preprocesar los datos con la nueva función
X_train_pca, X_test_pca, scaler, pca = preprocesar_datos(X_train, X_test)

# Entrenar el modelo
lda_model = entrenar_modelo(X_train_pca, y_train)

# Evaluar el modelo con un umbral ajustado
evaluar_con_umbral(lda_model, X_test_pca, y_test, umbral=0.48)


Rendimiento promedio (validación cruzada): 0.6248875155851901
Evaluación con umbral = 0.48
              precision    recall  f1-score   support

           0       0.60      0.61      0.60       466
           1       0.59      0.58      0.59       454

    accuracy                           0.59       920
   macro avg       0.59      0.59      0.59       920
weighted avg       0.59      0.59      0.59       920



In [14]:
# Preprocesar el dataset completo (X)
def preprocesar_completo(X, scaler, pca):
    # Hacer One-Hot Encoding del dataset completo
    X_encoded = pd.get_dummies(X, drop_first=True)
    
    # Estandarizar usando el scaler entrenado
    X_scaled = scaler.transform(X_encoded)
    
    # Aplicar PCA usando el modelo PCA entrenado
    X_pca = pca.transform(X_scaled)
    
    return X_pca

# Aplicar el preprocesamiento al dataset completo
X_completo_pca = preprocesar_completo(X, scaler, pca)

# Hacer predicciones con el modelo entrenado en el dataset completo
y_pred_completo = lda_model.predict(X_completo_pca)

# Evaluar el rendimiento del modelo en el dataset completo
print("Reporte de clasificación para el dataset completo:")
print(classification_report(y, y_pred_completo))


Reporte de clasificación para el dataset completo:
              precision    recall  f1-score   support

           0       0.61      0.67      0.64      1533
           1       0.63      0.57      0.60      1533

    accuracy                           0.62      3066
   macro avg       0.62      0.62      0.62      3066
weighted avg       0.62      0.62      0.62      3066

